In [2]:
import pandas as pd
import numpy as np

In [3]:
df_games=pd.read_parquet('ETL_games_clean')
df_reviews = pd.read_parquet('ETL_user_reviews_clean')

In [4]:
df_games.head(2)

,genres,title,release_date,price,item_id,developer
0,Action,Orbitality,2017,9.99,753100.0,Wonderbox Games
1,Indie,Castle In The Darkness,2015,9.99,262960.0,"Matt Kap,Nicalis, Inc."


In [5]:
df_reviews.head(2)

,user_id,item_id,reviews_helpful,reviews_recommend,reviews_date,sentiment_analysis
0,76561197970982479,1250,No ratings yet,True,2011-11-05,2
1,js41637,251610,15 of 20 people (75%) found this review helpful,True,2014-06-24,2


In [6]:
#De df_games se tomara la columna title y se usa item_id para el merge
df_games_name =df_games[['title','item_id']]

#tomo las columnas necesarias de df_reviews
df_need_reviews = df_reviews[['sentiment_analysis','item_id']]

In [7]:
#averiguo el tipo de columna que necesito que tengan en comun mis dataframe para poder hacer un merge
type_item_id = df_games_name['item_id'].dtype
print(type_item_id)

type_item_id = df_need_reviews['item_id'].dtype
print(type_item_id)



float64
object


In [8]:
# Convertir la columna 'columna_object' a tipo float
df_need_reviews['item_id'] = df_need_reviews['item_id'].astype(float)

C:\Users\Nathaly\AppData\Local\Temp\ipykernel_9768\541754622.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_need_reviews['item_id'] = df_need_reviews['item_id'].astype(float)


In [9]:
#hago un merge para quedar con un solo dataframe
df_modelo = pd.merge(df_need_reviews,df_games_name, on='item_id',how='inner')

df_modelo

,sentiment_analysis,item_id,title
0,2,1250.0,Killing Floor
1,1,211420.0,DARK SOULS™: Prepare To Die™ Edition
2,1,211420.0,DARK SOULS™: Prepare To Die™ Edition
3,0,249130.0,LEGO® Marvel™ Super Heroes
4,0,249130.0,LEGO® Marvel™ Super Heroes
...,...,...,...
77042,2,265630.0,Fistful of Frags
77043,2,304050.0,Trove
77044,2,304050.0,Trove
77045,2,304050.0,Trove


In [10]:
df_modelo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77047 entries, 0 to 77046
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   sentiment_analysis  77047 non-null  int64  
 1   item_id             77047 non-null  float64
 2   title               77047 non-null  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 1.8+ MB


In [11]:
# Verificar duplicados en todo el DataFrame
hay_duplicados = df_modelo.duplicated().any()

if hay_duplicados:
    print("El DataFrame tiene filas duplicadas.")
else:
    print("El DataFrame no tiene filas duplicadas.")

El DataFrame tiene filas duplicadas.


In [12]:
#eliminacion de duplicados
df_modelo.drop_duplicates(subset=['item_id', 'title'],inplace=True)

In [13]:
def procesar_sentimiento(valor):
    if valor == 1:
        return 1
    
    if valor == 2:
        return 2
    else:
        return None

# Crear la nueva columna 'rating' aplicando la función a la columna 'analisis_sentimiento'
df_modelo['rating'] = df_modelo['sentiment_analysis'].apply(procesar_sentimiento)

In [14]:
df_modelo.head(2)

,sentiment_analysis,item_id,title,rating
0,2,1250.0,Killing Floor,2.0
1,1,211420.0,DARK SOULS™: Prepare To Die™ Edition,1.0


In [15]:
# Verificar si hay elementos 'NaN' en el DataFrame
if df_modelo.isna().any().any():
    print("El DataFrame contiene elementos 'NaN'.")
else:
    print("El DataFrame no contiene elementos 'NaN'.")

El DataFrame contiene elementos 'NaN'.


In [16]:
#Eliminar los NaN para no tener conflitos
df_modelo.dropna(inplace=True)

In [17]:
df_modelo

,sentiment_analysis,item_id,title,rating
0,2,1250.0,Killing Floor,2.0
1,1,211420.0,DARK SOULS™: Prepare To Die™ Edition,1.0
5,1,4000.0,Garry's Mod,1.0
6,1,42910.0,Magicka,1.0
8,2,232090.0,Killing Floor 2,2.0
...,...,...,...,...
76639,2,305960.0,Dragon: The Game,2.0
76647,2,203650.0,Sonic the Hedgehog 4 - Episode II,2.0
76729,1,22610.0,Alien Breed: Impact,1.0
76927,2,16600.0,Trials 2: Second Edition,2.0


In [18]:
#Elimino a 	sentiment_analysis
df_modelo.drop(columns=['sentiment_analysis'], inplace=True)

In [56]:
float_columns = df_modelo.select_dtypes(include=['float64']).columns
df_modelo[float_columns] = df_modelo[float_columns].astype(int)

In [57]:
df_modelo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1647 entries, 0 to 76941
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   item_id  1647 non-null   int32 
 1   title    1647 non-null   object
 2   rating   1647 non-null   int32 
dtypes: int32(2), object(1)
memory usage: 38.6+ KB


In [58]:
# Se finaliza guardando el dataframe recomendación en formato comprimido en parquet
df_modelo.to_parquet('df_recomendacion', engine='pyarrow')